<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO:** poprawić to, żeby robiło *git push* albo *git clone link* w zależności od potrzeby

In [ ]:
%run fetch_repo.py

In [1]:
# Setup dependencies for the project
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Launch labelImg
!labelImg

In [6]:
import torch

rand = torch.rand(5, 3)
print(rand)

tensor([[0.6911, 0.6480, 0.2945],
        [0.3720, 0.2621, 0.3562],
        [0.0833, 0.8154, 0.2560],
        [0.9047, 0.7416, 0.1554],
        [0.7222, 0.8233, 0.4087]])


Dżem dobry
:)